In [1]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

print(grandparent_dir)
sys.path.append(grandparent_dir)

c:\Workspace\Code\Repos\pieofcode1\ticketier


In [22]:
"""Load support tickets data from CSV into PostgreSQL database."""
import csv
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
from src.core.pg_helper import *
from pprint import pprint

load_dotenv()

True

In [3]:
# search_similar_issues("WiFi keeps dropping")
search_similar_issues("Slow internet connection")

[{'ticket_id': 'PRB8412802',
  'created': '2025-10-19',
  'status': 'Resolved',
  'issue_summary': 'Slow Wi-Fi speed',
  'priority': 'Medium',
  'affected_service': 'Internet',
  'customer_impact': 'Unable to make calls',
  'assigned_team': 'Tier 1 Support',
  'similarity_score': 0.9397939651897602},
 {'ticket_id': 'INC8731198',
  'created': '2025-10-14',
  'status': 'Open',
  'issue_summary': 'Slow Wi-Fi speed',
  'priority': 'Medium',
  'affected_service': 'Home Security',
  'customer_impact': 'Security system offline',
  'assigned_team': 'Field Operations',
  'similarity_score': 0.9397939651897602},
 {'ticket_id': 'INC9768155',
  'created': '2025-09-15',
  'status': 'Open',
  'issue_summary': 'Slow Wi-Fi speed',
  'priority': 'Low',
  'affected_service': 'Voice',
  'customer_impact': 'Security system offline',
  'assigned_team': 'Field Operations',
  'similarity_score': 0.9397939651897602},
 {'ticket_id': 'CHG8900439',
  'created': '2025-11-15',
  'status': 'In Progress',
  'issue_s

### Create the agent with tool access
We created the agent in the Azure AI Foundry project and added the Postgres tools needed to query the database.

In [11]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
# from azure.ai.projects.models import FilePurpose, CodeInterpreterTool, ListSortOrder, MessageRole
from azure.ai.projects.models import PromptAgentDefinition, FunctionTool, Tool
from openai.types.responses.response_input_param import FunctionCallOutput, ResponseInputParam
from datetime import datetime

In [13]:
# [START tool_declaration]
tool = FunctionTool(
    name="search_similar_issues",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The issue description to search for similar issues",
            },
        },
        "required": ["query"],
        "additionalProperties": False,
    },
    description="Search for similar issues in the support tickets database based on the provided issue description.",
    strict=True,
)
# [END tool_declaration]

In [14]:
instructions = """
You are a helpful legal assistant who can retrieve information about service tickets from postgres database.
Select the following tools to answer the user's question:
- search_similar_issues(issue_description: str, limit: int = 5) -> List[Dict]
    : Searches for similar issues in the support tickets database based on the provided issue description.

Once you have the relevant information, reason over that to provide optimal priority, affected service, customer impact, and assigned team.
Respond in JSON format.
"""

In [ ]:
with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"], credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):
    agent = project_client.agents.create_version(
        agent_name="Ticket-Classifier-Agent",
        definition=PromptAgentDefinition(
            model=os.environ["GPT_MODEL_DEPLOYMENT_NAME"],
            instructions=instructions,
            tools=[tool],
        ),
    )

    # Prompt the model with tools defined
    response = openai_client.responses.create(
        input="Slow internet connection",
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )
    print(f"Response output: {response.output_text}")

    input_list: ResponseInputParam = []
    # Process function calls
    for item in response.output:
        if item.type == "function_call":
            if item.name == "search_similar_issues":
                # Execute the function logic for search_similar_issues
                similar_issues = search_similar_issues(**json.loads(item.arguments))

                # Provide function call results to the model
                input_list.append(
                    FunctionCallOutput(
                        type="function_call_output",
                        call_id=item.call_id,
                        output=json.dumps({"similar_issues": similar_issues}),
                    )
                )

    print("Final input:")
    print(input_list)

    response = openai_client.responses.create(
        input=input_list,
        previous_response_id=response.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

    # The model should be able to give a response!
    print("Final output:")
    print("\n" + response.output_text)

Response output: 
Final input:
[{'type': 'function_call_output', 'call_id': 'call_BpuUYlDeWlL379mCgqSPGSWn', 'output': '{"similar_issues": [{"ticket_id": "PRB8412802", "created": "2025-10-19", "status": "Resolved", "issue_summary": "Slow Wi-Fi speed", "priority": "Medium", "affected_service": "Internet", "customer_impact": "Unable to make calls", "assigned_team": "Tier 1 Support", "similarity_score": 0.9397939651897602}, {"ticket_id": "INC8731198", "created": "2025-10-14", "status": "Open", "issue_summary": "Slow Wi-Fi speed", "priority": "Medium", "affected_service": "Home Security", "customer_impact": "Security system offline", "assigned_team": "Field Operations", "similarity_score": 0.9397939651897602}, {"ticket_id": "INC9768155", "created": "2025-09-15", "status": "Open", "issue_summary": "Slow Wi-Fi speed", "priority": "Low", "affected_service": "Voice", "customer_impact": "Security system offline", "assigned_team": "Field Operations", "similarity_score": 0.9397939651897602}, {"ti

In [24]:
response.conversation

In [25]:
pprint(response.model_dump())

{'agent': {'name': 'Ticket-Classifier-Agent',
           'type': 'agent_id',
           'version': '3'},
 'background': False,
 'conversation': None,
 'created_at': 1763565573.0,
 'error': None,
 'id': 'resp_04192667624b0edd00691de005adf081948086c9d7f3d6a694',
 'incomplete_details': None,
 'instructions': '\n'
                 'You are a helpful legal assistant who can retrieve '
                 'information about service tickets from postgres database.\n'
                 "Select the following tools to answer the user's question:\n"
                 '- search_similar_issues(issue_description: str, limit: int = '
                 '5) -> List[Dict]\n'
                 '    : Searches for similar issues in the support tickets '
                 'database based on the provided issue description.\n'
                 '\n'
                 'Once you have the relevant information, reason over that to '
                 'provide optimal priority, affected service, customer impact, '
         

In [18]:
dir(response)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_on_complete__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__